# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

## Откройте и изучите файл

In [1]:
#импорт билиотек
import random
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

RANDOM_STATE = 12345

In [2]:
#импорт данных
try:
    data = pd.read_csv('/datasets/users_behavior.csv')
except:
    data = pd.read_csv('C:\\datasets/users_behavior.csv')

In [3]:
#выводим первые 5 строк датафрейма
data.head()

In [4]:
#основная информация о датафрейме
data.info()

In [5]:
#основные статистики датафрейма
data.describe().T

**Вывод**
<br>Действительно, предобработка данных не требуется. Данные выглядят довольно однородными - без аномалий и резких выбросов.

## Разбейте данные на выборки

In [6]:
#определим целевой признак и прочие
target = data['is_ultra']
features = data.drop('is_ultra', axis=1)

In [7]:
#разобьем все данные на обучающую, валидационную и тестовую выборки (60/20/20)
features_train, features_vnt, target_train, target_vnt = train_test_split(
    features, target, test_size=0.40, random_state=RANDOM_STATE, stratify=target)
features_valid, features_test, target_valid, target_test = train_test_split(
    features_vnt, target_vnt, test_size=0.5, random_state=RANDOM_STATE, stratify=target_vnt)

In [8]:
#проверяем результат
features_train.shape

In [9]:
features_valid.shape

In [10]:
features_train.shape[0]/data.shape[0]

**Вывод**
<br>Имеем 3 выборки: обучающую (60% исходных данных), валидационную (20%) и тестовую (20%).

## Исследуйте модели

**Дерево решений**

Рассмотрим точность предсказаний модели дерева решений с различным параметром max_depth:

In [11]:
dtc_accuracy = []
train_accuracy =[]
for depth in range(1,41):
    dtc_model = DecisionTreeClassifier(random_state=RANDOM_STATE, max_depth=depth)
    dtc_model.fit(features_train, target_train)
    predictions_valid = dtc_model.predict(features_valid)
    dtc_accuracy.append(accuracy_score(target_valid, predictions_valid))
    train_accuracy.append(accuracy_score(target_test, dtc_model.predict(features_test)))
#строим график по полученным значениям
plt.plot([i+1 for i in range(len(dtc_accuracy))], dtc_accuracy);
plt.plot([i+1 for i in range(len(dtc_accuracy))], train_accuracy);
plt.plot([i+1 for i in range(len(dtc_accuracy))], [0.75 for i in range(len(dtc_accuracy))], '--');
plt.suptitle('Decision Tree')
plt.legend (('validation data', 'train data', 'min accuracy'))
plt.xlabel('max_depth')
plt.ylabel('accuracy')
plt.grid()

Точность предсказания на валидационой выборке растет нестабильно, иногда приближаясь к пороговому значению (0.75). Только с глубины >= 5 можно говорить о хорошей точности предсказания. При больших значениях параметра max_depth видим переобучение.

**Случайный лес**

Рассмотрим точность предсказаний модели случайного леса с различным параметром n_estimators (1-40) и max_depth (1-20):

In [12]:
%%time
rfc_accuracy = pd.DataFrame(columns=[i for i in range(1,21)], index=[i for i in range(1,41)])
for depth in range(1, 21):
    for est in range(1, 41):
        rfc_model = RandomForestClassifier(random_state=RANDOM_STATE, n_estimators=est, max_depth=depth)
        rfc_model.fit(features_train, target_train)
        rfc_accuracy.loc[est, depth] = rfc_model.score(features_valid, target_valid)

In [13]:
rfc_accuracy = rfc_accuracy.astype(float)

In [14]:
sns.heatmap(rfc_accuracy);

Оптимальные значения получаем с параметрами n_estimators=39 и max_depth=9

**Логистическая регрессия**

Рассмотрим точность предсказаний модели логистической регрессии с различными гиперпараметрами:

In [15]:
%%time
solvers = ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']
for i in range(len(solvers)):
    lr_model = LogisticRegression(random_state=RANDOM_STATE, solver=solvers[i], max_iter=10000)
    lr_model.fit(features_train, target_train)
    predictions_valid = lr_model.predict(features_valid)
    print('solver =', solvers[i], 'accuracy =', f'{accuracy_score(target_valid, predictions_valid):.3f}')

Как видно, здесь мы не можем перешагнуть порог точности в 0.75 ни в одном из вариантов.

**Вывод**
<br>Наилучшие значения точности на валидационной выборке достигаются с моделью случайного леса  параметрами n_estimators=39 и max_depth=9

## Проверьте модель на тестовой выборке

Обучаем модель на объединенной обучающей + валидационной выборках. Проверяем выбранную модель на тестовой выборке:

In [16]:
rfc_model = RandomForestClassifier(random_state=RANDOM_STATE, n_estimators=39, max_depth=9)
rfc_model.fit(pd.concat([features_train, features_valid], ignore_index=True, axis=0, sort=False),
              pd.concat([target_train, target_valid], ignore_index=True, axis=0, sort=False))
print('accuracy:', f"{rfc_model.score(features_test, target_test):.3f}")

**Вывод**
<br>На тестовой выборке получено значение точности выше 0.75. С помощью RandomForestClassifier мы добились точности 0.829 с гиперпараметрами n_estimators=39 и max_depth=9.

## (бонус) Проверьте модели на адекватность

In [17]:
from sklearn.dummy import DummyClassifier

In [18]:
dummy_model = DummyClassifier(strategy='most_frequent', random_state=RANDOM_STATE)
dummy_model.fit(pd.concat([features_train, features_valid], ignore_index=True, axis=0, sort=False),
                pd.concat([target_train, target_valid], ignore_index=True, axis=0, sort=False))
print('accuracy:', f"{dummy_model.score(features_test, target_test):.3f}")

**Вывод**
<br>0.694 - Базовая "точность" константной модели. Все используемые модели по своей точности ее превосходят, включая оптимальную.